## Dataloader

## Create index DataFrame

In [1]:
from dataset import load_pdbbind_data_index

index_location = '/home/longyuxi/Documents/mount/pdbbind-dataset/index/INDEX_refined_data.2020'
index = load_pdbbind_data_index(index_location)

## For debug purposes

# index = index[:10]
# index

### Calculating the persistence diagram of each of the coordinates and storing them

In [ ]:
from pathlib import Path

# for each item, calculate the persistence homology of it
output_folder = 'computed_homologies'

# get pdb file for both things
base_folder = Path(index_location).parent.parent / 'refined-set'

# list of commands to be run by GNU parallel
python_interpreter = '/home/longyuxi/miniconda3/envs/donaldlab/bin/python'
homology_calculator_location = '/home/longyuxi/Documents/protein-conformation-topology/calculate_homology.py'
pairwise_opposition_homology_calculator_location = '/home/longyuxi/Documents/protein-conformation-topology/calculate_pairwise_opposition_homologies_binned.py'

# for any of the files, if they don't exist, calculate homology for it
commands = ''
for idx, row in index.iterrows():
    pdb_name = row['PDB code']
    diagram_output_folder = Path(output_folder) / pdb_name
    diagram_output_folder.mkdir(parents=True, exist_ok=True)
    pocket_pdb = base_folder / pdb_name / f'{pdb_name}_pocket.pdb'
    protein_pdb = base_folder / pdb_name / f'{pdb_name}_protein.pdb'
    ligand_mol2 = base_folder / pdb_name / f'{pdb_name}_ligand.mol2'

    # # protein
    # if not (diagram_output_folder / 'protein.pckl').is_file():
    #     commands += f'{python_interpreter} {homology_calculator_location} --input {str(protein_pdb)} --output {str(diagram_output_folder / "protein.pckl")}\n'

    # # pocket
    # if not (diagram_output_folder / 'pocket.pckl').is_file():
    #     commands += f'{python_interpreter} {homology_calculator_location} --input {str(pocket_pdb)} --output {str(diagram_output_folder / "pocket.pckl")}\n'


    # # ligand
    # if not (diagram_output_folder / 'ligand.pckl').is_file():
    #     commands += f'{python_interpreter} {homology_calculator_location} --input {str(ligand_mol2)} --output {str(diagram_output_folder / "ligand.pckl")}\n'

    # pairwise opposition homology
    if not (diagram_output_folder / 'pairwise_opposition.pckl').is_file():
        commands += f'{python_interpreter} {pairwise_opposition_homology_calculator_location} --protein {str(protein_pdb)} --ligand {str(ligand_mol2)} --output {str(diagram_output_folder / "pairwise_opposition.pckl")}\n'



with open('cmds.tmp', 'w') as cf:
    cf.write(commands)

# Then run the following in terminal
#   time parallel -j 10 --eta < cmds.tmp

### Vietoris-Rips persistence with opposition disatnce

##### Opposition distance calculation
$d_{op}(a_i, a_j) = $ Euclidean distance if one atom comes from protein and the other from ligand, else $\infty$.

Trick: append each atom coordinates with 0 for protein, 1 for ligand.

**Question**: Does it matter where the proteins and ligands are placed relatively?

In [ ]:
import pickle

v = pickle.load(open('/home/longyuxi/Documents/protein-conformation-topology/computed_homologies/2w8w/pairwise_opposition.pckl', 'rb'))